In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import RobertaTokenizerFast, RobertaModel, RobertaForSequenceClassification

In [3]:
# model = RobertaModel.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained('roberta-base')
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 512)

# tokenizer(" Hello world")['input_ids']

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [5]:
# dataset['train']['text'][25]
def tokenize_fn(text):
    tokenized =  tokenizer(text['text'], 
                   truncation = True, 
                   max_length = 512, 
                   return_tensors='pt',
                  )
    
    return tokenized


In [6]:
import datasets
train_data_imbd, test_data_imbd = datasets.load_dataset('imdb', split =['train', 'test'])
train_data_imbd = train_data_imbd.shuffle(seed=42)
test_data_imbd = test_data_imbd.shuffle(seed=42)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
# train_data_imbd['label'][3]

In [16]:
train_len = 2000
train_dataset_imbd = train_data_imbd.select(list(range(train_len)))
test_dataset_imbd = test_data_imbd.select(list(range(train_len//10)))
# train_dataset_imbd = train_data_imbd
# test_dataset_imbd = test_data_imbd
train_dataset_imbd
# train_data_2['label'][200]

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

### Finetune

In [25]:
# train_data_2['text'][2]
def tokenize_fn(batched_text):
    output = tokenizer(batched_text['text'], padding=True, truncation=True, return_tensors='pt')
    return output

def rename_column(example):
    example['deleteme'] = example['label']
    del example['label']
    return example

train_imbd = train_dataset_imbd.map(tokenize_fn, batched = True)
test_imbd = test_dataset_imbd.map(tokenize_fn, batched = True)

train_imbd = train_imbd.map(rename_column)
test_imbd = test_imbd.map(rename_column)

train_imbd.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_imbd.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
# train_imbd.set_format(type='torch', columns=['input_ids', 'attention_mask', 'deleteme'])
# test_imbd.set_format(type='torch', columns=['input_ids', 'attention_mask', 'deleteme'])

In [26]:
# type(train_imbd['input_ids'][3])
train_imbd

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'deleteme'],
    num_rows: 2000
})

In [27]:
# define accuracy metrics
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
from datasets import load_metric

metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)


/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [28]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = '/kaggle/working/results',
    num_train_epochs=3,    
    evaluation_strategy="steps",
    eval_steps=25,
    logging_strategy="steps",
    logging_steps=25,
    logging_dir='/kaggle/working/logs',
    report_to = 'none',
)

In [29]:
# instantiate the trainer class and check for available devices
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_imbd,
    eval_dataset=test_imbd, 
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


'cuda'

In [30]:
trainer.train()

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.